In [16]:
!/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/bin/python -m pip install matplotlib.pyplot

ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
You should consider upgrading via the '/Users/sooryasrajan/.local/share/virtualenvs/LightweightBlockCipherClassifier-y2MFm9FQ/bin/python -m pip install --upgrade pip' command.


In [15]:
import json
from neo4j import GraphDatabase
import pandas as pd
import networkx as nx

In [3]:
with open('processed_data_mvd.json', 'r') as file:
    data = json.load(file)

In [5]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password"))

In [6]:
df = pd.DataFrame(data)

In [7]:
df.columns

Index(['id', 'normalized_name', 'start', 'end', 'text', 'labels',
       'document_id', 'parent_id', 'publishdate', 'title', 'docsource'],
      dtype='object')

In [19]:
grouped = df.groupby('document_id')

In [52]:
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

In [53]:
count = 0
# iterate grouped pandas
for name, group in grouped:
    # shuffle data
    # shuffle panda frame
    group = group.sample(frac=1)
    if count == 0 or count == 1:
        count += 1
        continue
    if count == 3:
        break

    # create a graph for each case
    G = nx.Graph()
    # add nodes to graph
    for index, row in group.iterrows():
        G.add_node(row['normalized_name'], type=row['labels'])
    # add edges to graph
    for i in range(len(group) - 1):
        current = group.iloc[i]['normalized_name']
        next_node = group.iloc[i + 1]['normalized_name']
        # add relationship label
        relationship = group.iloc[i]['labels'] + '-' + group.iloc[i + 1]['labels']
        G.add_edge(current, next_node, relationship=relationship)
    # add nodes to neo4j
    with driver.session() as session:
        caseName = group.iloc[0]['docsource']
        # make case node color orange
        # create a central node for each case with the case name and make it orange
        session.run("CREATE (n:Node {name: $name, type: $type})", name=caseName, type='CASE')

        # add nodes to neo4j
        caseCount = 0
        for node in G.nodes:
            print(node)
            caseCount = caseCount + 1
            session.run("CREATE (n:Node {name: $name, type: $type})", name=node, type=G.nodes[node]['type'])

        #connect all nodes to the above created node
        for node in G.nodes:
            session.run("MATCH (a:Node),(b:Node) WHERE a.name = $from_node AND b.name = $to_node CREATE (a)-["
                        "r:RELATIONSHIP {relationship: $relationship}]->(b)", from_node=node, to_node=caseName,
                        relationship='CASE')

    count += 1

2
